<a href="https://colab.research.google.com/github/joe3gold/Google-Chrome-Google-Contacts-Sync/blob/master/WhisperGoogleDrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

If you're looking at this on GitHub and new to Python Notebooks or Colab, click the Google Colab badge above 👆

#OpenAI Whisper and Google Drive integration for transcribing audio files

📺 Getting started video: https://youtu.be/yVLhG4-7Sj4

###This notebook will transcribe all the audio files in a Google Drive folder

*Note: This requires giving the application permission to connect to your drive. Only you will have access to the contents of your drive, but please read the warnings carefully.*

This notebook application:
1. Connects to your Google Drive when you give it permission.
2. Creates a WhisperAudio folder and two subfolders (ProcessedAudio and TextFiles.)
3. When you run the application it will search for all the audio files (.mp3 and .m4a) in your WhisperAudio folder, transcribe them and then move the file to /ProcessedAudio and save the transcript to /TextFiles.

###**For faster performance set your runtime to "GPU"**
*Click on "Runtime" in the menu and click "Change runtime type". Select "GPU".*


**Note: If you add a new file after running this application you'll need to remount the drive in step 1 to make them searchable**

##1. Install the required code libraries

In [2]:
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg
!pip install librosa

import whisper
import time
import librosa
import soundfile as sf
import re
import os

# model = whisper.load_model("tiny.en")
# model = whisper.load_model("base.en")
model = whisper.load_model("small.en") # load the small model
# model = whisper.load_model("medium.en")
# model = whisper.load_model("large")

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-dgviu6yc
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-dgviu6yc
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 http:/

##2. Allow access to your Google Drive and add new folders

In [3]:
# Connect Google Drive
from google.colab import drive
drive.mount("/content/drive", force_remount=True) # This will prompt for authorization.

# This will create the WhisperAudio files if they don't exist.
folders =  ["WhisperAudio/", "WhisperAudio/ProcessedAudio/", "WhisperAudio/TextFiles/"]
for folder in folders:
  path = "/content/drive/MyDrive/" + folder
  if not os.path.exists(path): # Create the folder if it does not exist
    os.mkdir(path)

Mounted at /content/drive


##3. Upload any audio files you want transcribed in the "WhisperAudio" folder in your Google Drive.

##4. Let the application search for new files and transcribe the audio files and save them to your Google Drive

In [ ]:
# Load all the audio file paths in a Google Drive folder
from google.colab import drive
drive.mount("/content/drive", force_remount=True) # This will prompt for authorization.

# Assuming the audio files are in a folder called "WhisperAudio" in the root of the drive
audio_folder = "/content/drive/MyDrive/WhisperAudio/"

# Get a list of all the file paths and names in the folder
import os
audio_files = []
audio_names = []
for file in os.listdir(audio_folder):
  if file.endswith(".m4a") or file.endswith(".mp3"):
    audio_files.append(audio_folder + file)
    audio_names.append(file)

for f in audio_files:
  print(f)

if len(audio_files) == 0:
  print("You have no files.")

# Loop through the audio files, split each audio file based on pauses in speech then transcribe them with Whisper.
for i, file in enumerate(audio_files): # For each audio file
  print(f"Processing {audio_names[i]}...")
  # Load the audio file and convert it to 16 kHz mono
  audio, sr = librosa.load(file, sr=16000, mono=True)
  # Detect pauses and split the audio. We use a threshold of -30 dB and a minimum pause length of 0.5 seconds.
  pauses = librosa.effects.split(audio, top_db=30, frame_length=2048, hop_length=128)
  # Transcribe each segment and concatenate the results
  transcription = ""
  for start, end in pauses: # For each segment
    segment = audio[start:end]
    # Save the segment as a temporary wav file
    temp_file = "temp.wav"
    sf.write(temp_file, segment, sr, subtype='PCM_16')
    if os.path.getsize(temp_file) > 10000:
      #continue
      # Transcribe the segment with Whisper
      result = model.transcribe(temp_file)
      text = result["text"]
      # Append the text to the transcription
      print(len(transcription.split(" ")), "words processed")
      transcription += text.strip() + " "
      # Delete the temporary file
      os.remove(temp_file)
  # Print the transcription
  print(f"Transcription of {audio_names[i]}:\n")
  print(transcription)
  print("\n")

  # Convert the spaces between sections into paragraph breaks and save the transcription as a txt document in the same folder as MyAudio.
  transcription = re.sub(r"\s\s+", "\n\n", transcription) # Replace multiple spaces with newlines
  text_file = audio_folder + "/TextFiles/" + audio_names[i][:-4] + ".txt" # Create the text file name
  with open(text_file, "w") as f: # Write the transcription to the text file
    f.write(transcription)
  print(f"Saved transcription as {text_file}")

# Move the audio files to "/content/drive/MyDrive/WhisperAudio/Processed"
import shutil
processed_folder = "/content/drive/MyDrive/WhisperAudio/ProcessedAudio/"
if not os.path.exists(processed_folder): # Create the folder if it does not exist
  os.mkdir(processed_folder)
for file in audio_files: # Move each audio file to the processed folder
  shutil.move(file, processed_folder + os.path.basename(file))
  print(f"Moved {file} to {processed_folder}")

Mounted at /content/drive
/content/drive/MyDrive/WhisperAudio/Voice Memo - 2025-09-10 02 59 51 - New Recording.m4a
/content/drive/MyDrive/WhisperAudio/Voice Memo - 2025-09-25 19 21 32 - Recording 4.m4a
/content/drive/MyDrive/WhisperAudio/Voice Memo - 2025-09-10 05 47 26 - Finally getting some crystal when OG got back!.m4a
/content/drive/MyDrive/WhisperAudio/Voice Memo - 2025-09-03 01 48 54 - 515 N La Brea Ave.m4a
/content/drive/MyDrive/WhisperAudio/Voice Memo - 2025-09-03 01 36 06 - Morry calling.m4a
/content/drive/MyDrive/WhisperAudio/Voice Memo - 2025-09-10 03 00 27 - Looking for Chrystal 3:00 am.m4a
/content/drive/MyDrive/WhisperAudio/Voice Memo - 2025-09-26 10 07 55 - Probation, Fairmount Tires, Jocelyn.m4a
Processing Voice Memo - 2025-09-10 02 59 51 - New Recording.m4a...


/tmp/ipython-input-2017401303.py:27: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(file, sr=16000, mono=True)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Transcription of Voice Memo - 2025-09-10 02 59 51 - New Recording.m4a:




Saved transcription as /content/drive/MyDrive/WhisperAudio//TextFiles/Voice Memo - 2025-09-10 02 59 51 - New Recording.txt
Processing Voice Memo - 2025-09-25 19 21 32 - Recording 4.m4a...


/tmp/ipython-input-2017401303.py:27: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(file, sr=16000, mono=True)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


1 words processed


...

/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


1142 words processed


/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


1144 words processed
Transcription of Voice Memo - 2025-09-10 03 00 27 - Looking for Chrystal 3:00 am.m4a:

  Thank you.         Thank you.  Thank you. Thank you.  Thank you. Thank you.      Thanks for watching! Thank you. Thanks for watching! Thanks for watching! Thank you. Thank you.       Thanks for watching! Thank you. Thanks for watching!         Okay.                      Yeah. I feel it. Thank you. You know what I think is causing it right now? Nobody's out of the street like you know you know anybody I know you're worth it.      Please, aye? Thank you.     Okay.          Thanks for watching! Thank you.  Thanks for watching! Thanks for watching!  Thank you. Thank you.   Thanks for watching! Thanks for watching!    Thanks for watching!  Thank you. Thanks for watching! Thanks for watching! Thank you. Thanks for watching! Thanks for watching!           you   Thank you. Thank you.  Thank you. Thank you. Thanks for watching! Thank you. Thanks for watching! Thanks for watching! Thanks

/tmp/ipython-input-2017401303.py:27: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(file, sr=16000, mono=True)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
